In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

import math

In [2]:
frame1=cv2.imread("LEFT1.jpg")
frame2=cv2.imread("RIGHT1.jpg")

frame1=cv2.resize(frame1,(640,640))
frame2=cv2.resize(frame2,(640,640))


In [3]:
def Oper (frame):
    frame_new=frame.copy()
    frame_1=cv2.cvtColor(frame_new,cv2.COLOR_BGR2HSV)
    lower_red = np.array([170,50,50])
    upper_red = np.array([180,255,255])
    frame_hsv = cv2.inRange(frame_1, lower_red, upper_red)
    #frameG=cv2.blur(frame1,(5,5))
    #frameC=cv2.Canny(frameG,130,200)
    contours,hierarchy=cv2.findContours(frame_hsv,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        cv2.drawContours(frame_new,[c],-1,(0,255,255),2)
        x, y, w, h = cv2.boundingRect(c)
        # draw a green rectangle to visualize the bounding rect
        cv2.rectangle(frame_new, (x, y), (x+w, y+h), (0, 255, 0), 2)
    return frame_new,frame_hsv,contours

In [4]:
def Tem_Match(image1,image2,contour1,contour2):
    img1=image1.copy()
    img2=image2.copy()
    img3=img2.copy()
    img1=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    row,col=img1.shape
    
    zero_mat=np.zeros((row,col),np.uint8)
    
    #img1=cv2.Canny(img1,100,200)
    #img2=cv2.Canny(img2,100,200)
    
    for c in contour1:
        x, y, w, h = cv2.boundingRect(c)
        temp1=img1[y:y+h,x:x+w]
        
       # methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
           # 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
        
        res = cv2.matchTemplate(img2[y:y+h,:],temp1,cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        top_left=max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        
        #euclid_dis= math.sqrt(pow((x-top_left[0]),2)+pow((y-top_left[1]),2))
        euclid_dis= math.sqrt(pow((x-top_left[0]),2)+pow((top_left[1]),2))
        euclid_dis = int(euclid_dis)
        
        zero_mat[top_left[1]+y:top_left[1]+y+h,top_left[0]:top_left[0]+w]=euclid_dis
        
        image2=cv2.putText(img3, "euclid dis", (top_left[0]+30, top_left[1]+y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)
        image2=cv2.putText(img3, str(euclid_dis), (top_left[0], top_left[1]+y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)

        #print(type(euclid_dis))
        #print(euclid_dis)
        #cv2.rectangle(img2,top_left, bottom_right, 255, 2)
        #cv2.imshow("LL",img2)
        #if cv2.waitKey(1000) 
        #cv2.waitKey(1)
        #cv2.destroyAllWindows()
    return zero_mat,image2

In [5]:
frame_new1,frame_hsv1,contours1=Oper(frame1)
frame_new2,frame_hsv2,contours2=Oper(frame2)
dis_mat,image2=Tem_Match(frame1,frame2,contours1,contours2)

cv2.imshow("detected_cont",frame_new1)
cv2.imshow("hsv_seg",frame_hsv1)
cv2.imshow("localized",image2)
cv2.imshow("dis",dis_mat)

cv2.waitKey(0)
cv2.destroyAllWindows()